In [ ]:
import conntility
import numpy
import pandas
import morphio
import os

import bluepysnap as bluepy
from conntility.subcellular import MorphologyPathDistanceCalculator

# CONFIGURATION

# Circuit config
# fn_circ = "/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/InterneuronRewiringO1/circuits/SSCx-HexO1-Release__O1_Schneider_Mizell_v2/circuit_config_reparam_tc.json"
fn_circ = "/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology/circuit_config.json"

# Name of the connectome to consider
connectome = "S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"
# Name of the node population to consider
population = 'S1nonbarrel_neurons'
# Radius of the population to consider, i.e. we only analyze neurons within this distance of the center
radius_to_use = 150
# For calculation of the inhibitory (synapse) in-degree:
## File to place results into
fn_out_degree = "Inhibition_synapse_counts.csv"
# For calculation of soma path distances of inhibitory synapses
## File to place results into
fn_out_pd = "Inhibition_path_distances.csv"
## Number of neurons per class to sample
neurons_per_class = 50

# BASIC SETUP
circ = bluepy.Circuit(fn_circ)

morph_dir = None
for _pop in circ.config["networks"]["nodes"]:
    if population in _pop["populations"]:
        morph_dir = _pop["populations"][population]["alternate_morphologies"]["neurolucida-asc"]
assert morph_dir is not None

# Loading all neurons and finding circuit center

In [ ]:
nrn = conntility.circuit_models.neuron_groups.load_neurons(circ, ["x", "y", "z", "ss_flat_x", "ss_flat_y", "morphology",
                                                                 "mtype", "layer", "synapse_class"],
                                                          node_population=population)
population_center = nrn[["ss_flat_x", "ss_flat_y"]].mean()

# Loading connectivity and counting synapses of each connection
Note that we load only the most central population as parameterized

In [ ]:
load_cfg = {
    "loading":{    
        "properties": ["x", "y", "z", "mtype", "layer", "synapse_class", "morphology", "ss_flat_x", "ss_flat_y"],
        "base_target": "hex_O1",
    },
    "filtering":[
        {
            "column": "ss_flat_x",
            "interval": [population_center["ss_flat_x"] - radius_to_use,
                         population_center["ss_flat_x"] + radius_to_use]
        },
        {
            "column": "ss_flat_y",
            "interval": [population_center["ss_flat_y"] - radius_to_use,
                         population_center["ss_flat_y"] + radius_to_use]
        }
    ]
}

# Note: "conductance" does not mean anything here, since we use "len" as the aggregation function.
# That is, we take the "conductance" values of all synapses in a connection, but then just take the length of that vector,
# i.e. we simply calculate the synapse count.
cmat = conntility.ConnectivityMatrix.from_bluepy(circ, load_config=load_cfg,
                                                connectome=connectome,
                                                edge_property="conductance", agg_func=len)


## Submatrix of only inhibitory connections
We filter the connection matrix.
The result still contains all neurons, but only inhibitory edges

We also load a numpy array of all inhbitory gids

In [ ]:
icmat = cmat.filter("synapse_class", side="row").eq("INH")
inh_gids = nrn.set_index("synapse_class").loc["INH"]["node_ids"].values

## Calculate inhibitory in-degrees and save
We use the filtered matrix to calculate total inhibitory in-degrees.
We then group the results by layer/synapse class and save

In [ ]:
i_indegree = numpy.array(icmat.matrix.sum(axis=0))[0]
icmat.add_vertex_property("inh_indegree", i_indegree)

i_indeg_per_class = pandas.concat([
    icmat.vertices.groupby(["layer", "synapse_class"])["inh_indegree"].mean(),
    icmat.vertices.groupby(["layer", "synapse_class"])["inh_indegree"].std()
], axis=1, keys=["mean", "std"])

i_indeg_per_class.reset_index().to_csv(fn_out_degree)

# Inhibitory synapse path distances
Path distances are a bit more expensive to calculate.
So we calculate them for only a smaller number of randomly sampled neurons per class

In [ ]:
neuron_samples = icmat.vertices.groupby(["synapse_class", "layer"])["node_ids"].apply(lambda _x: numpy.random.choice(_x.values, neurons_per_class))
neuron_samples.head()

### Load synapses, calculate path distances....

In [ ]:
path_distance_res = {}

for nrn_class, lst_gids in neuron_samples.items():
    print(nrn_class)
    if numpy.any(numpy.isnan(lst_gids)):
        continue
    res = [] # where we put all samples for the indicated post-synaptic class
    for gid in lst_gids:
        # Load morphology
        morph_fn = icmat.vertices.set_index("node_ids").loc[gid]["morphology"]
        morph_fn = os.path.join(morph_dir, morph_fn) + ".asc"
        m = morphio.Morphology(morph_fn)

        # Load anatomical positions of synapses
        edges = circ.edges[connectome]
        syns = edges.afferent_edges(gid, properties=[bluepy.sonata_constants.Edge.SOURCE_NODE_ID,
                                                                  "afferent_section_id", "afferent_segment_id", "afferent_segment_offset"])
        # Only inhibitory synapses...
        syn_is_inh = numpy.in1d(syns[bluepy.sonata_constants.Edge.SOURCE_NODE_ID], inh_gids)
        syns = syns.loc[syn_is_inh]
        # The anatomical soma location is section #0, segment #0
        soma = pandas.DataFrame({"afferent_section_id": [0], "afferent_segment_id": [0], "afferent_segment_offset": [0]})

        # Calculate path distances from all inhibitory synapses (syns) to the soma
        M = MorphologyPathDistanceCalculator(m)
        D = M.path_distances(syns, soma)

        res.extend(D[:, 0])
    path_distance_res[nrn_class] = (numpy.mean(res), numpy.std(res))



Turn into DataFrame and save

In [ ]:
path_distance_res = \
pandas.DataFrame(path_distance_res).transpose().reset_index().rename(columns={0: "mean", 1: "std",
                                                                             "level_0": "synapse_class",
                                                                             "level_1": "layer"})
path_distance_res.to_csv(fn_out_pd)